<sub>**Note**: To run all the code at once, click the <span style="letter-spacing:-3px;">**&blacktriangleright;&blacktriangleright;**</span> &nbsp;&nbsp;button. To run code blocks one at a time, type *shift+enter* or click the **&blacktriangleright;** button.</sub>

# USD Fundamentals

## Learning Objectives

Through a series of videos and hands-on code samples, this Jupyter notebook will demonstrate some of USD's fundamental features. 

Upon completion you will have a working understanding of the USD scene description format and how to leverage its features in a collaborative pipeline.

* [USD Format Overview](#USD-Format-Overview)
* [How to use USD Sublayers](#Sublayers)
* [How to use USD References](#References)
* [How to use USD Variants](#Variants)


## Setup

Since visualizing USD content typically requires installing software on a workstation, we have created tools to assist in previewing content created during this lab. While these tools only surface a subset of everything USD has to offer, we hope this will provide a more wholesome learning experience, and encourage you to continue this learning journey towards [USD](https://usd.nvidia.com) and [Omniverse](https://developer.nvidia.com/nvidia-omniverse-platform).

In [1]:
from nvidia_utils.display import display_usd

We have also created a `/usd_fundamentals` folder under the `/usd_files` directory, in order to hold the generated USD files for this lab. Feel free to navigate to this location using the file viewer on the left-hand side of the Jupyter interface should you want to save or edit your content. In order to download a file, right click it and select "Download".

## USD Format Overview

In [2]:
from utils.display import display_video
display_video("1_k9eix8fk")

ModuleNotFoundError: No module named 'utils'

First, let's discuss some things about USD that are common with other file formats.

A USD file, which is often called a _layer_, is a lot like other graphics files: a single file can contain models, meshes, lights, shaders, and other useful graphic primitives.

A USD file can have multiple representations. The most common are:
 * `.USDC` files (also called _crate_ files), compressed and memory-mapped, fast-access file format.
 * `.USDA` files are human-readable files. This can be useful to understand what is in a USD file without rendering it, and for debugging and experimenting.
 * `.USDZ` files are ZIP archives containing a USD layer and its supporting files like textures, for rendering.

In this guide, we will be using the USDA representation, as it is the most appropriate to visualize the structure of USD scenes.

### First steps into USD

The following _cube.usda_ file is a very simple USD file which only contains a cube. We have defined a transform called "World", and a Mesh called "Cube". Each of these are an example of a [Prim](https://graphics.pixar.com/usd/docs/USD-Glossary.html#USDGlossary-Prim), an abstract container for USD content. `Prim`s are defined with the `def` keyword, and can contain data expressed properties and attributes.

If you look at the text representation of the mesh, you will find it has all the properties you might expect a mesh to have like vertex counts, vertices, normals, and display colors.

Feel free to render the cube, and to modify any of its properties to see how they affect the scene:

In [3]:
import os
print(os.getcwd())

C:\Users\jason\OneDrive\Desktop\Projects\USD Practice\NVIDIA_USD_Practice


In [6]:
%%file usd_files/usd_fundamentals/cube.usda
#usda 1.0

def Xform "World"
{
  def Mesh "Cube"
  {
    # Cube geometry:
    float3[] extent = [(-0.5, -0.5, -0.5), (0.5, 0.5, 0.5)]
    int[] faceVertexCounts = [4, 4, 4, 4, 4, 4]
    int[] faceVertexIndices = [0, 1, 3, 2, 2, 3, 5, 4, 4, 5, 7, 6, 6, 7, 1, 0, 1, 7, 5, 3, 6, 0, 2, 4]
    point3f[] points = [(-0.5, -0.5, 0.5), (0.5, -0.5, 0.5), (-0.5, 0.5, 0.5), (0.5, 0.5, 0.5), (-0.5, 0.5, -0.5), (0.5, 0.5, -0.5), (-0.5, -0.5, -0.5), (0.5, -0.5, -0.5)]
    
    # Surface color:
    color3f[] primvars:displayColor = [(0.463, 0.725, 0.0)]
    
    # Coordinates:
    double3 xformOp:translate = (1.0, 0.0, 0.0)
    double3 xformOp:rotateXYZ = (0.0, 45.0, 0.0)
    float3 xformOp:scale = (1.0, 1.0, 1.0)
    uniform token[] xformOpOrder = ["xformOp:translate", "xformOp:rotateXYZ", "xformOp:scale"]
  }
}

Writing usd_files/usd_fundamentals/cube.usda


FileNotFoundError: [Errno 2] No such file or directory: 'usd_files/usd_fundamentals/cube.usda'

### Render the scene

Let's see how this content looks like in 3D:

In [ ]:
display_usd('usd_files/usd_fundamentals/cube.usda')

**TODO**: Try modifying [usd_files/usd_fundamentals/cube.usda](usd_files/usd_fundamentals/cube.usda) and rerendering the scene above. Can you change the color, rotation, or scale?

## USD Composition Arcs

In [2]:
display_video("1_z21wbyrb")

What makes USD unique?

So far, we've only talked about things that are easily accomplished in other kinds of graphics files. One of the most interesting features of USD is the idea of layer composition.

Layer composition refers to the ways USD files can be combined together. There are a number of ways you can combine USD files, and these are usually referred to as _composition arcs_.

There are 6 kinds of composition arcs in USD:

 1. **Sublayers**: similar to layers in Photoshop, sublayers in USD define a stack of USD layers. A stack is in strength order, and sublayers can override each other. If two sublayers have values for the same attribute, the strongest sublayer wins.
 2. **References**: used to add new items into a scene. For instance, if you have a chair model, and you want 4 chairs around a table, you could add 4 references to the chair model.
 3. **Payloads**: like a reference that can be loaded in a deferred way. You can open a scene and choose which item of the scene to load in memory.
 4. **Variants**: used to switch between items in a collection. For example, if the chair model had a weather version, a wood version and an Eames version, a variant would let us choose between them.
 5. **Inherits**: works in a similar fashion to a class in an object-oriented programming language. You define a class of items, like "Humans", which all have an `eyeColor` property. All human Prims could inherit from that class, and automatically have the shared `eyeColor` property.
 6. **Specializes**: behaves in a similar way to _Inherits_, but with subtle differences that may be too specific for the scope of this brief overview.

## Sublayers

In [3]:
display_video("1_4isqdezo")

The following USDA file demonstrates the features of the sublayer composition arc, by:
 * Referencing the _cube.usda_ file created earlier, on top of which a new _blue-cube.usda_ file is stacked.
 * The _blue-cube.usda_ file being on top of the sublayer stack, the properties it has in common with the "weaker" _cube.usda_ file carry more weight.
 * For demonstration purposes, we use this fact to force the cube to have a blue `displayColor` property by using an `over` on the mesh to overwrite any previously-set values.

Feel free to go back to the _cube.usda_ file and edit any of its properties, or to overwrite any additional properties to see the effect of stacking layers on the composition of the final render:

In [ ]:
%%file usd_files/usd_fundamentals/blue-cube.usda
#usda 1.0

over Xform "World" {
  over Mesh "Cube" {
    # Try uncommenting additional properties to see the effect on the final
    # render, or adding new ones and see how it affects the scene:
    color3f[] primvars:displayColor = [(0.0, 0.0, 1.0)]
    # double3 xformOp:rotateXYZ = (45.0, 45.0, 0.0)
    # float3 xformOp:scale = (1.0, 2.0, 3.0)
  }
}

In [ ]:
%%file usd_files/usd_fundamentals/cube-model.usda
#usda 1.0

(
  subLayers = [
    @./blue-cube.usda@,
    @./cube.usda@
  ]
)

### Render the scene

In [ ]:
display_usd('usd_files/usd_fundamentals/cube-model.usda')

**TODO**: Try uncommenting lines 9 and 10 from [usd_files/usd_fundamentals/blue-cube.usda](usd_files/usd_fundamentals/blue-cube.usda) and rerender the scene using the code cell above. Can you spot the difference?

## References

In [4]:
display_video("1_y7qw6fhh")

References are for combining assets together.

Let's say we want to create a scene with a collection of models. For our purposes, we'll make a scene with 4 of our cube models in it.

One way to do that in USD is to use references. Here, we'll be adding 4 references to the _cube-model.usda_ file we created earlier. 

One nice side-effect of this setup is that the _cube-model.usda_ asset is only loaded into memory once. As you can see, we have defined 3 Prims, each one referencing the USDA file we created earlier.

A few things worth mentioning:
 1. Notice that the "CubeOne" `def` does not specify that the Prim is a `Mesh`. This is because USD is able to infer the type from the object it references.
 2. If you look at the reference path, notice that it has a file path for _cube-model.usda_ in the current directory, but also a Prim path to `/World/Cube`. This indicates that we want to *only* reference the "Cube" Prim and not any other Prim which may also be contained in the file.
 3. Notice that we set a translation on the 3 last cubes to slide them down the X axis to separate them in 3D space. By doing this in the text, we've overridden the transform on the cube that we brought in, which was located at the origin. You could override any other attributes in the same way, like the color, changing vertices, or even adding new geometry or assigning new shaders.

In [ ]:
%%file usd_files/usd_fundamentals/my-scene.usda
#usda 1.0

def Xform "World" {

  def "CubeOne" (references = @./cube-model.usda@</World/Cube>) {
    
  }

  def "CubeTwo" (references = [@./cube-model.usda@</World/Cube>]) {
    # Try changing the position of one of the references:
    double3 xformOp:translate = (4.0, 0.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
  }

  def "CubeThree" (references = [@./cube-model.usda@</World/Cube>]) {
    # ... or its scale:
    float3 xformOp:scale = (2.0, 2.0, 2.0)
    double3 xformOp:translate = (2.25, 0.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:scale", "xformOp:translate"]
  }

  def "CubeFour" (references = [@./cube-model.usda@</World/Cube>]) {
    # ... or even multiple properties at once:
    float3 xformOp:scale = (1.5, 2.5, 4.0)
    double3 xformOp:translate = (4.5, 0.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:scale", "xformOp:translate"]
    color3f[] primvars:displayColor = [(0.8, 0.2, 0.5)]
  }
}

### Render the scene

In [ ]:
display_usd('usd_files/usd_fundamentals/my-scene.usda')

**TODO**: This scene has four cubes, but only three are clearly visible. Can you alter [usd_files/usd_fundamentals/my-scene.usda](usd_files/usd_fundamentals/my-scene.usda) to make all four cubes clearly visible? If you've been experimenting with the previous USD files, you may have already done so.

## Variants

In [5]:
display_video("1_jpkegw1q")

Variants are a clever way in which properties of an asset can be toggled between different states, or "flavors". This can include a variety of use cases in which assets only differ by a few of their properties.

Examples of this can include:
 * Models of t-shirts, which only differ by `displayColor`.
 * Models of billboards in the background of a shot, which only differ by the texture of the sponsors with the highest bid.
 * Spaceship models, which only differ by their geometry to represent the level of damage they might have sustained during a battle scene.

The following example demonstrates a simplistic example of variants, where the display color of the cube can be toggled between `red`, `green` or `blue`. Try changing the `shadingVariant` and rerendering the asset to see how the affect the final render:

In [ ]:
%%file usd_files/usd_fundamentals/cube-variants.usda
#usda 1.0

def Xform "World"
{
  def Xform "Object"(
    variants = {
      # Try changing the shading variant to either "red", "green" or "blue":
      string shadingVariant = "green"
    }
    prepend variantSets = "shadingVariant"
  )
  {
    def "Cube" (references = @./cube-model.usda@</World/Cube>)
    {
      # Coordinates:
      double3 xformOp:translate = (0.0, 0.0, 0.0)
      double3 xformOp:rotateXYZ = (0.0, 45.0, 0.0)
      float3 xformOp:scale = (1.0, 1.0, 1.0)
      uniform token[] xformOpOrder = ["xformOp:translate", "xformOp:rotateXYZ", "xformOp:scale"]
     
      # Colors (set by specifying the "shadingVariant" on the "Cube" above):
      color3f[] primvars:displayColor
    }

    variantSet "shadingVariant" = {
      "red" {
        over "Cube" {
          color3f[] primvars:displayColor = [(1.0, 0.0, 0.0)]
        }
      }

      "green" {
        over "Cube" {
          color3f[] primvars:displayColor = [(0.0, 1.0, 0.0)]
        }
      }

      "blue" {
        over "Cube" {
          color3f[] primvars:displayColor = [(0.0, 0.0, 1.0)]
        }
      }
    }
  }
}

### Render the scene

In [ ]:
display_usd('usd_files/usd_fundamentals/cube-variants.usda')

**TODO**: Try changing `shadingVariant` on line 9 of [usd_files/usd_fundamentals/cube-variants.usda](usd_files/usd_fundamentals/cube-variants.usda) to either `red` or `blue`. Want an extra challenge? Try adding another color variant.

## Recap: USD Composition Arcs

In [5]:
display_video("1_3d4zsmce")

**Congratulations on completing this notebook!**

## Next steps

Now that you've gained some exposure to the fundamental concepts of USD, we encourage you to continue on your learning journey with additional resources:
 * See the [Advanced USD](02_Advanced_USD.ipynb) training to learn how these concepts are used in production, using Python.
 * Learn more about [the features that USD offers](https://usd.nvidia.com).
 * Download [Omniverse](https://developer.nvidia.com/nvidia-omniverse-platform) to see how it can let you express your creativity:

In [6]:
display_video("1_83x1r2o2")

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>